## Running EO4Atlantic Services

EO4Atlantic services may be executed locally outside of the [platform](http://portal.eo4a.science). This is currently achieved by means of Jupyter notebooks, which leverage eo4a.development module functions provided by the service framework, **and is the recommended method for local service development prior to deployment in the platform.** TODO: add user guide link.

This notebook demonstrates how to execute the example services provided in the [EO4Atlantic service development repository](https://github.com/parameterspace-ie/eo4a-service-development), and may be used as a starting point for executing other (new) services.

In [5]:
# Required imports from service development module.
from eo4a.development import create_wps, describe_service, list_services, monitor_execution, reload_services

As described in the user guide TODO, each EO4Atlantic service is hosted as an [OGC Web Processing Service (WPS)][wps]. When creating the development container (TODO), any [WPS][wps] definitions found in `$EO4A_SERVICES_DIR` will be made available. The following demonstrates how to list the available services.

[wps]: http://www.opengeospatial.org/standards/wps

In [6]:
wps = create_wps() # This reference is instantiated first, and used in most subsequent calls.
reload_services(wps) # This can be used to reload services when local code changes are made.
list_services(wps)

Processes reloaded successfully:
	merge_shapefiles
	sentinel2-rgb

Errors:


########## Services: ###########

Merge Shapefiles (id: merge_shapefiles)
----------------------------------------
Merge multiple shapefiles into a single file.


Sentinel-2 RGB (id: sentinel2-rgb)
----------------------------------------
Example service that generates composite RGB rasters from Sentinel-2 products, using bands 4, 3, and 2 respectively by default.
However, alternative bands may also be used, such as the false color (R=8, G=4, B=3).




Full details of individual services, including their [WPS][wps] input and output parameters, can be achieved as follows (the details are loaded from the corresponding `wps.py` module):

In [4]:
describe_service('sentinel2-rgb', wps)

Sentinel-2 RGB (id: sentinel2-rgb)
----------------------------------------
Example service that generates composite RGB rasters from Sentinel-2 products, using bands 4, 3, and 2 respectively by default.
However, alternative bands may also be used, such as the false color (R=8, G=4, B=3).


---- Inputs ----
	 identifier=s2_product_dir, title=Sentinel-2 product directory, abstract=Contains one or more Sentinel-2 products., data type=string
 Any value allowed
	 Default Value: None 
	 minOccurs=1, maxOccurs=1
	 identifier=r_band, title=R band, abstract=R band number (default=4).,, data type=integer
 Any value allowed
	 Default Value: 4 
	 minOccurs=0, maxOccurs=1
	 identifier=g_band, title=G band, abstract=G band number (default=3).,, data type=integer
 Any value allowed
	 Default Value: 3 
	 minOccurs=0, maxOccurs=1
	 identifier=b_band, title=B band, abstract=B band number (default=2).,, data type=integer
 Any value allowed
	 Default Value: 2 
	 minOccurs=0, maxOccurs=1
	 identifier=reso

In [4]:
wps = create_wps()
reload_services(wps)
inputs = [
    ('datasetname', "/opt/eo4a/services/eo4a-eotools/gdaltools/tests/data/raster_1k_1k.tif"),
    ('json', 'false'),
    ('mm', 'true'),
    ('stats', 'true'),
    ('approx_stats', 'true'),
    ('hist', 'true'),
    ('nogcp', 'true'),
    ('nomd', 'false'),
    ('norat', 'true'),
    ('noct', 'true'),
    ('checksum', 'true'),
    ('listmdd', 'true'),
    ('mdd', 'all'),
    ('nofl', 'true'),
    ('proj4', 'true'),
]
execution = wps.execute('gdalinfo', inputs)
print('Command output:')
print(execution.processOutputs[0].data[0])

Processes reloaded successfully:
	merge_shapefiles
	sentinel2-rgb
	portal:PublishRaster
	cmems
	data:cedasentinel
	data:scihub
	gdalinfo
	gdalwarp
	correction:6s
	landcover

Errors:

Command output:
Driver: GTiff/GeoTIFF
Files: /opt/eo4a/services/eo4a-eotools/gdaltools/tests/data/raster_1k_1k.tif
Size is 110, 110
Coordinate System is:
PROJCS["WGS 84 / UTM zone 29N",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",-9],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORIT